# Fake News Classifier using Bidirectional LSTM RNN

Dataset link: https://www.kaggle.com/c/fake-news/data?select=train.csv

We will be doing Sentiment Analysis prediction.

## Importing dataset

In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('train.csv', on_bad_lines='warn', engine='python')

<ipython-input-5-994837006473>:1: ParserWarning: Skipping line 4650: unexpected end of data

  df = pd.read_csv('train.csv', on_bad_lines='warn', engine='python')


In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


## Checking for null values

In [7]:
df.isna().sum()

,0
id,0
title,128
author,452
text,11
label,0


In [8]:
df.shape

(4648, 5)

## Dropping null values as its text datatype

In [9]:
df.dropna()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
4643,4643,The Hyperinflationary Death Watch,The Doc,Home » Headlines » Finance News » The Hyperinf...,1
4644,4644,Taxpayer Funded Peer Lord Kerr Calls Taxpayers...,dailouk,Home | World | Taxpayer Funded Peer Lord Kerr ...,1
4645,4645,Kidnapped Border Patrol Agent Was Hacked with ...,Brandon Darby &amp Ildefonso Ortiz,Suspects in the kidnapping and attack of a U. ...,0
4646,4646,NRA to Campaign Against Red State Democrats wh...,AWR Hawkins,With a vote on the confirmation of Supreme Cou...,0


In [10]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


## Getting the Independent features (X)

In [11]:
X = df.drop('label', axis=1)

In [24]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


## Getting the depndent feature

In [12]:
y = df['label']

## Checking whether dataset is balanced or not

In [13]:
y.value_counts()

,count
label,
1,2361
0,2287


We have good number of both output classes values. So its balanced.

## Importing required libraries from Tensorflow and keras

In [14]:
import tensorflow as tf

In [15]:
tf.__version__

'2.18.0'

In [17]:
from tensorflow.keras.layers import Embedding

from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential

from tensorflow.keras.preprocessing.text import one_hot

from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


## Specifying vocabulary size

In [18]:
voc_size = 5000

## Copying X into messages variable

In [19]:
messages = X.copy()

In [20]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
4643,4643,The Hyperinflationary Death Watch,The Doc,Home » Headlines » Finance News » The Hyperinf...
4644,4644,Taxpayer Funded Peer Lord Kerr Calls Taxpayers...,dailouk,Home | World | Taxpayer Funded Peer Lord Kerr ...
4645,4645,Kidnapped Border Patrol Agent Was Hacked with ...,Brandon Darby &amp Ildefonso Ortiz,Suspects in the kidnapping and attack of a U. ...
4646,4646,NRA to Campaign Against Red State Democrats wh...,AWR Hawkins,With a vote on the confirmation of Supreme Cou...


## Preprocessing text using nltk library

In [22]:
import nltk
import re

from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
messages.reset_index(inplace=True)

In [30]:
stemmer = PorterStemmer() # creating an object of Porter Stemmer class

pre_processed_messages = []

for i in range(len(messages)):
    # Convert the title to string before applying re.sub
    message = str(messages['title'][i])
    message = re.sub('[^a-zA-Z]',' ',message) # removing special characters
    message = message.lower() # lowering the sentences

    message = message.split() # splitting the words in the review
    message = [stemmer.stem(word) for word in message if not (word) in set(stopwords.words('english'))] # removing stopwords
    message = ' '.join(message) # joining to make the sentence look back to normal

    # adding to the preprocessed sentences list
    pre_processed_messages.append(message)

In [31]:
pre_processed_messages

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'excerpt draft script donald trump q ampa black church pastor new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find hom

## Converting into one hot encoding representation

In [35]:
onehot_representation = [one_hot(message, voc_size) for message in pre_processed_messages]

In [36]:
print(onehot_representation)

[[1535, 477, 4814, 4483, 1827, 150, 596, 2144, 4447, 597], [580, 2122, 2603, 3235, 2499, 3889, 4363], [349, 91, 2829, 4910], [2999, 3772, 2119, 4380, 1205, 923], [3171, 2499, 4125, 393, 3830, 4305, 2499, 2095, 2125, 4957], [1763, 3905, 652, 2782, 2689, 3088, 575, 3192, 2535, 3034, 1336, 1779, 1203, 4818, 4363], [4107, 4107, 981, 1951, 2702, 2091, 218, 3114, 89, 3351, 1366, 3558], [2440, 1938, 4681, 1822, 4195, 1980, 3147, 2216, 4873, 2092, 964], [1676, 4371, 2835, 2289, 3088, 2955, 2204, 3218, 1086, 2964, 4873, 2092, 964], [3834, 1429, 3552, 2044, 4703, 2956, 3088, 2750, 4873, 2092, 964], [3987, 65, 2692, 3166, 1501, 3934, 737, 1864, 3088, 2564], [4612, 1629, 1148, 4672, 2822, 2359, 1067, 4634], [4167, 4707, 763, 3766, 3446, 4254, 3167, 1301, 3495, 3437, 3301], [4380, 4666, 1827, 3934, 3088, 4703], [300, 1009, 840, 4553, 2301, 344, 4760, 2393, 4517], [2632, 3611, 1791, 2526, 1534, 1472, 3356, 4873, 2092, 964], [623, 1094, 1772, 1106, 1307, 4873, 2092, 964], [400, 1076, 3645, 1556, 3626

## Applying padding

In [38]:
sent_length = 20 # Maximum sentence length

padded_messages = pad_sequences(onehot_representation, padding='pre', maxlen=sent_length)

padded_messages

array([[   0,    0,    0, ..., 2144, 4447,  597],
       [   0,    0,    0, ..., 2499, 3889, 4363],
       [   0,    0,    0, ...,   91, 2829, 4910],
       ...,
       [   0,    0,    0, ..., 2144,  162, 3567],
       [   0,    0,    0, ...,   37, 2125, 4363],
       [   0,    0,    0, ..., 3863, 3673, 1086]], dtype=int32)

## Creating model

In [39]:
embedding_vector_features = 40 # vector representation

model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))

model.add(Bidirectional(LSTM(100))) # Bidirectional LSTM - 100 neurons forward and 100 neurons backward.

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


## Converting our features and labels into arrays

In [40]:
import numpy as np
X_final = np.array(padded_messages)
y_final = np.array(y)

## Train Test Split

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Training the model

In [42]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = 10, batch_size=32)

Epoch 1/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.7300 - loss: 0.5400 - val_accuracy: 0.9061 - val_loss: 0.2486
Epoch 2/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9270 - loss: 0.1893 - val_accuracy: 0.9159 - val_loss: 0.2218
Epoch 3/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9818 - loss: 0.0650 - val_accuracy: 0.9068 - val_loss: 0.3323
Epoch 4/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9948 - loss: 0.0205 - val_accuracy: 0.8983 - val_loss: 0.3761
Epoch 5/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9966 - loss: 0.0113 - val_accuracy: 0.9003 - val_loss: 0.4714
Epoch 6/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9993 - loss: 0.0025 - val_accuracy: 0.9022 - val_loss: 0.5211
Epoch 7/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 1.0000 - loss: 0.0022 - val_accuracy: 0.8931 - val_loss: 0.5636
Epoch 8/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 1.0000 - loss: 5.7036e-04 - val_accuracy: 0.9022 -

## Predicting the model

In [43]:
y_pred = model.predict(X_test)

48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [44]:
y_pred = np.where(y_pred > 0.5, 1, 0)

## Checking accuracy

In [45]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [46]:
accuracy_score(y_test, y_pred)

0.8976531942633638

In [47]:
confusion_matrix(y_test, y_pred)

array([[681,  86],
       [ 71, 696]])